<a href="https://colab.research.google.com/github/cjfghk5697/AjouDeeplearning_Compete/blob/main/Inference(TTA%2BEnssanble).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/Test image2
!unzip -qq "/content/drive/MyDrive/multi-mnist zipfile/test_dirty_mnist_2nd.zip"

/content/drive/MyDrive/Test image2


In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [ ]:
#test Dataset 정의
transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

batch_size=32
sample_submission = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")
testset = MnistDataset('/content/drive/MyDrive/Test image2', '/content/drive/MyDrive/sample_submission.csv', transforms_test)


In [ ]:
from torchvision.models.efficientnet import efficientnet_b0
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = efficientnet_b0(pretrained=True)
        self.dropout=nn.Dropout(p=0.2)
        self.classifier = nn.Linear(1000, 26)
        self.silu=nn.SiLU(inplace=False)
    def forward(self, x):
        #x = F.relu(self.model(x))
        x = self.model(x) 
        x = self.dropout(x)
        x = self.silu(x) 
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

path = "/content/drive/MyDrive/models/"
model_list=[]

model.load_state_dict(torch.load('/content/drive/MyDrive/models/_06012 eff0.pth'), strict=False)

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-cf984f9c.pth


  0%|          | 0.00/47.2M [00:00<?, ?B/s]

<All keys matched successfully>

In [ ]:
model = MnistModel()
model.to(device)# gpu에 모델 할당
import copy
best_models = []
model.load_state_dict(torch.load('1st fold'))
best_models.append(copy.deepcopy(model))
model.load_state_dict(torch.load('2nd fold'))
best_models.append(copy.deepcopy(model))
model.load_state_dict(torch.load('3th fold'))
best_models.append(copy.deepcopy(model))
model.load_state_dict(torch.load('4th fold'))
best_models.append(copy.deepcopy(model))
model.load_state_dict(torch.load('5th fold'))
best_models.append(copy.deepcopy(model))

In [ ]:
transforms_test1 = transforms.Compose([
                                      
    transforms.RandomHorizontalFlip(p=0),
    transforms.RandomVerticalFlip(p=0),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test2 = transforms.Compose([
                                      
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomVerticalFlip(p=0),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test3 = transforms.Compose([
                                      
    transforms.RandomHorizontalFlip(p=0),
    transforms.RandomVerticalFlip(p=1),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test4 = transforms.Compose([
                                      
    transforms.RandomHorizontalFlip(p=0),
    transforms.RandomVerticalFlip(p=0),
    ImageNetPolicy(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

#test Dataset 정의
sample_submission = pd.read_csv("/content/sample_submission.csv")
test_df = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

test_dataset1 = MnistDataset('/content/drive/MyDrive/Test image2', '/content/drive/MyDrive/sample_submission.csv', transforms_test,transforms_test1)
test_dataset2 = MnistDataset('/content/drive/MyDrive/Test image2', '/content/drive/MyDrive/sample_submission.csv', transforms_test,transforms_test2)
test_dataset3 = MnistDataset('/content/drive/MyDrive/Test image2', '/content/drive/MyDrive/sample_submission.csv', transforms_test,transforms_test3)
test_dataset4 = MnistDataset('/content/drive/MyDrive/Test image2', '/content/drive/MyDrive/sample_submission.csv', transforms_test,transforms_test4)


batch_size = 32
test_data_loader1 = DataLoader(
    test_dataset1,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 4,
    drop_last = False
)
test_data_loader2 = DataLoader(
    test_dataset2,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 4,
    drop_last = False
)
test_data_loader3 = DataLoader(
    test_dataset3,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 4,
    drop_last = False
)
test_data_loader4 = DataLoader(
    test_dataset4,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 4,
    drop_last = False
)

In [ ]:
predictions_list = []
prediction_df = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

# 4개의 data_loader를 모두 사용 (TTA)
for test_data_loader in [test_data_loader1, test_data_loader2, test_data_loader3, test_data_loader4]:
  # 5개의 fold마다 가장 좋은 모델을 이용하여 예측
  for model in best_models:
      prediction_array = np.zeros([prediction_df.shape[0],
                                prediction_df.shape[1] -1])      
      for idx, sample in enumerate(test_data_loader):
      for i, (images, targets) in enumerate(test_loader):

          with torch.no_grad():

              model.eval()
              images = images.to(device)
              probs  = model(images)
              probs = torch.sigmoid(probs).cpu().detach().numpy()

              prediction_array[idx*batch_size: (idx+1)*batch_size ,:]\
                          = probs.astype(np.float32)
                          
      # 채널을 하나 추가하여 list에 append
      predictions_list.append(prediction_array[...,np.newaxis])
  
# axis = 2를 기준으로 평균 (TTA, folds for each SEED)
predictions_array = np.concatenate(predictions_list, axis = 2)
predictions_mean = predictions_array.mean(axis = 2)

# 평균 값이 0.5보다 클 경우 1 작으면 0
predictions_mean = (predictions_mean > 0.5) * 1
predictions_mean

sample_submission = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")
sample_submission.iloc[:,1:] = predictions_mean
sample_submission.to_csv(f'perfect_efficientnetb7_rs{SEED}.csv", index = False) # 경로 수정 필요
sample_submission

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
test_loader = DataLoader(testset, batch_size=32, num_workers=4)
model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('/content/drive/MyDrive/models/baseline_prediction_0613 1.csv', index=False)